# Deep Universal Regular Conditional Expectations:

---
This implements the universal deep neural model of $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$ [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.
4. Learn the probability distribution that the unique strong solution to the rough SDE with uniformly Lipschitz drivers driven by a factional Brownian motion with Hurst exponent $H \in [\frac1{2},1)$:
$$
X_t^x = x + \int_0^t \alpha(s,X_s^x)ds + \int_0^t \beta(s,X_s^x)dB_s^H
$$
belongs, at time $t=1$, to a ball about the initial point $x$ of random radius given by an independant exponential random-variable with shape parameter $\lambda=2$
5. Train a DNN to predict the returns of bitcoin with GD.  Since this has random initialization then each prediction of a given $x$ is stochastic...We learn the distribution of this conditional RV (conditioned on x in the input space).
$$
Y_x \triangleq \hat{f}_{\theta_{T}}(x), \qquad \theta_{(t+1)}\triangleq \theta_{(t)} + \lambda \sum_{x \in \mathbb{X}} \nabla_{\theta}\|\hat{f}_{\theta_t}(x) - f(x)\|, \qquad \theta_0 \sim N_d(0,1);
$$
$T\in \mathbb{N}$ is a fixed number of "SGD" iterations (typically identified by cross-validation on a single SGD trajectory for a single initialization) and where $\theta \in \mathbb{R}^{(d_{J}+1)+\sum_{j=0}^{J-1} (d_{j+1}d_j + 1)}$ and $d_j$ is the dimension of the "bias" vector $b_j$ defining each layer of the DNN with layer dimensions:
$$
\hat{f}_{\theta}(x)\triangleq A^{(J)}x^{(J)} + b^{(J)},\qquad x^{(j+1)}\triangleq \sigma\bullet A^{j}x^{(j)} + b^{j},\qquad x^{(0)}\triangleq x
.
$$

#### Mode:
Software/Hardware Testing or Real-Deal?

In [1]:
trial_run = True

### Simulation Method:

In [2]:
# # Random DNN
# f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# # Random DNN internal noise
# f_unknown_mode = "DNN_with_Random_Weights"
Depth_Bayesian_DNN = 1
width = 5

# # Random Dropout applied to trained DNN
f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.1

# GD with Randomized Input
# f_unknown_mode = "GD_with_randomized_input"
GD_epochs = 2

# SDE with fractional Driver
# f_unknown_mode = "Rough_SDE"
N_Euler_Steps = 10**2
Hurst_Exponent = 0.5

## Problem Dimension

In [3]:
problem_dim = 50

## Note: *Why the procedure is so computationally efficient*?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [4]:
train_test_ratio = .2
N_train_size = 10**2

Monte-Carlo Paramters

In [5]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**2

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [6]:
# Hyper-parameters of Cover
delta = 0.01
Proportion_per_cluster = .75

## Dependencies and Auxiliary Script(s)

In [7]:
# %run Loader.ipynb
exec(open('Loader.py').read())
# Load Packages/Modules
exec(open('Init_Dump.py').read())
import time as time #<- Note sure why...but its always seems to need 'its own special loading...'

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready
Deep Feature Builder - Ready


# Simulate or Parse Data

In [8]:
# %run Data_Simulator_and_Parser.ipynb
exec(open('Data_Simulator_and_Parser.py').read())

 16%|█▌        | 16/100 [00:00<00:00, 158.02it/s]

---------------------------------------
Beginning Data-Parsing/Simulation Phase
---------------------------------------
Deciding on Which Simulator/Parser To Load
Setting/Defining: Internal Parameters
Deciding on Which Type of Data to Get/Simulate
Simulating Output Data for given input data


100%|██████████| 20/20 [00:00<00:00, 137.48it/s]

----------------------------------
Done Data-Parsing/Simulation Phase
----------------------------------


# Run Main:

In [9]:
print("------------------------------")
print("Running script for main model!")
print("------------------------------")
# %run Universal_Measure_Valued_Networks_Backend.ipynb
exec(open('Universal_Measure_Valued_Networks_Backend.py').read())

print("------------------------------------")
print("Done: Running script for main model!")
print("------------------------------------")

------------------------------
Running script for main model!
------------------------------


100%|██████████| 75/75 [00:00<00:00, 12359.45it/s]

Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.1s finished


Epoch 1/5
13/13 [==============================] - 0s 1ms/step - loss: 4.3268 - accuracy: 0.0500
Epoch 2/5
13/13 [==============================] - 0s 2ms/step - loss: 4.3055 - accuracy: 0.0600
Epoch 3/5
13/13 [==============================] - 0s 2ms/step - loss: 4.2904 - accuracy: 0.0600
Epoch 4/5
13/13 [==============================] - 0s 2ms/step - loss: 4.2763 - accuracy: 0.0600
Epoch 5/5
3/3 [==============================] - 0s 908us/step


  0%|          | 0/100 [00:00<?, ?it/s]

Training Classifer Portion of Type Model: Done!
#--------------------#
 Get Training Error(s)
#--------------------#


100%|██████████| 20/20 [00:00<00:00, 475.95it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#
#------------------------#
 Get Testing Error(s): END
#------------------------#
                                       DNM  MC-Oracle
W1-95L                            0.000016   0.000000
W1                                0.000016   0.000000
W1-95R                            0.000016   0.000000
M-95L                             0.000136   0.000136
M                                 0.000136   0.000136
M-95R                             0.000136   0.000136
N_Par                          1335.000000   0.000000
Train_Time                        5.530573   0.855534
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000
------------------------------------
Done: Running script for main model!
------------------------------------


---
# Run: All Benchmarks

## 1) *Pointmass Benchmark(s)*
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [10]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


100%|██████████| 20/20 [00:00<00:00, 2360.13it/s]

---------------------
Training: ENET - Done
---------------------
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                       DNM  MC-Oracle          ENET
W1-95L                            0.000016   0.000000  1.602694e-08
W1                                0.000016   0.000000  1.602694e-08
W1-95R                            0.000016   0.000000  1.602694e-08
M-95L                             0.000136   0.000136  1.265976e-04
M                                 0.000136   0.000136  1.265976e-04
M-95R                             0.000136   0.000136  1.265976e-04
N_Par                          1335.000000   0.000000  2.000000e+02
Train_Time                        5.530573   0.855534  1.619880e+09
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04
-----------------
Training


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0501s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished
100%|██████████| 20/20 [00:00<00:00, 2818.85it/s]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000016   0.000000  1.602694e-08   
W1                                0.000016   0.000000  1.602694e-08   
W1-95R                            0.000016   0.000000  1.602694e-08   
M-95L                             0.000136   0.000136  1.265976e-04   
M                                 0.000136   0.000136  1.265976e-04   
M-95R                             0.000136   0.000136  1.265976e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                                     KRidge  
W1-95L         

...

Epoch 1/5
13/13 [==============================] - 0s 2ms/step - loss: 0.0587 - mse: 0.0035 - mae: 0.0587 - mape: 58264992.0000
Epoch 2/5
13/13 [==============================] - 0s 2ms/step - loss: 0.0566 - mse: 0.0033 - mae: 0.0566 - mape: 56174248.0000
Epoch 3/5
13/13 [==============================] - 0s 2ms/step - loss: 0.0544 - mse: 0.0031 - mae: 0.0544 - mape: 54093536.0000
Epoch 4/5
13/13 [==============================] - 0s 2ms/step - loss: 0.0523 - mse: 0.0028 - mae: 0.0523 - mape: 51985072.0000
Epoch 5/5
3/3 [==============================] - 0s 801us/step


  0%|          | 0/100 [00:00<?, ?it/s]

#------------#
 Get Error(s) 
#------------#


100%|██████████| 20/20 [00:00<00:00, 3168.50it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000016   0.000000  1.602694e-08   
W1                                0.000016   0.000000  1.602694e-08   
W1-95R                            0.000016   0.000000  1.602694e-08   
M-95L                             0.000136   0.000136  1.265976e-04   
M                                 0.000136   0.000136  1.265976e-04   
M-95R                             0.000136   0.000136  1.265976e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                                     KRidge          GBRF         DNN  
W1-95L                         1.5

# Summary of Point-Mass Regression Models

#### Training Model Facts

In [11]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000020   0.000000  1.602694e-08   
W1                                0.000035   0.000000  1.496441e-05   
W1-95R                            0.000078   0.000000  4.486117e-05   
M-95L                             0.000170   0.000168  1.265976e-04   
M                                 0.000295   0.000295  2.506632e-04   
M-95R                             0.000543   0.000664  4.987944e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                                     KRidge          GBRF         DNN  
W1-95L                         1.580490e-08  6.791680e-08    0.002347  
W1                             1.496441e-05  1.365428e-05    0.002520  
W1-95R                         4.486162e-05  4.078659e-05    0.002712  
M

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,0.000020,0.000000,1.602694e-08,1.580490e-08,6.791680e-08,0.002347
W1,0.000035,0.000000,1.496441e-05,1.496441e-05,1.365428e-05,0.002520
W1-95R,0.000078,0.000000,4.486117e-05,4.486162e-05,4.078659e-05,0.002712
M-95L,0.000170,0.000168,1.265976e-04,1.257175e-04,1.616148e-04,0.046939
M,0.000295,0.000295,2.506632e-04,2.498008e-04,2.233496e-04,0.049023
M-95R,0.000543,0.000664,4.987944e-04,4.979672e-04,3.150803e-04,0.050634
N_Par,1335.000000,0.000000,2.000000e+02,0.000000e+00,1.109000e+05,521.000000
Train_Time,5.530573,0.855534,1.619880e+09,5.926316e-01,8.619890e-01,5.066507
Test_Time/MC-Oracle_Test_Time,0.461736,1.000000,4.379825e-04,3.304335e-03,5.246057e-03,0.344872


#### Testing Model Facts

In [12]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000016   0.000000  1.602694e-08   
W1                                0.000016   0.000000  1.602694e-08   
W1-95R                            0.000016   0.000000  1.602694e-08   
M-95L                             0.000136   0.000136  1.265976e-04   
M                                 0.000136   0.000136  1.265976e-04   
M-95R                             0.000136   0.000136  1.265976e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                                     KRidge          GBRF         DNN  
W1-95L                         1.580490e-08  1.093142e-08    0.002389  
W1                             1.580490e-08  1.149926e-07    0.002797  
W1-95R                         1.580490e-08  3.208878e-07    0.003277  
M

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,0.000016,0.000000,1.602694e-08,1.580490e-08,1.093142e-08,0.002389
W1,0.000016,0.000000,1.602694e-08,1.580490e-08,1.149926e-07,0.002797
W1-95R,0.000016,0.000000,1.602694e-08,1.580490e-08,3.208878e-07,0.003277
M-95L,0.000136,0.000136,1.265976e-04,1.257175e-04,9.070589e-05,0.045869
M,0.000136,0.000136,1.265976e-04,1.257175e-04,1.967334e-04,0.051925
M-95R,0.000136,0.000136,1.265976e-04,1.257175e-04,3.444595e-04,0.056059
N_Par,1335.000000,0.000000,2.000000e+02,0.000000e+00,1.109000e+05,521.000000
Train_Time,5.530573,0.855534,1.619880e+09,5.926316e-01,8.619890e-01,5.066507
Test_Time/MC-Oracle_Test_Time,0.461736,1.000000,4.379825e-04,3.304335e-03,5.246057e-03,0.344872


## 2) *Gaussian Benchmarks*

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\Sigma}(x)\hat{\Sigma}^{\top})\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to 
(2\pi)^{-\frac{d}{2}}\det(\hat{\Sigma}(x))^{-\frac{1}{2}} \, e^{ -\frac{1}{2}(\cdot - \hat{\mu}(x))^{{{\!\mathsf{T}}}} \hat{\Sigma}(x)^{-1}(\cdot - \hat{\mu}(x)) } \mu \in \mathcal{G}_d\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

Examples of this type of architecture are especially prevalent in uncertainty quantification; see ([Deep Ensembles](https://arxiv.org/abs/1612.01474)] or [NOMU: Neural Optimization-based Model Uncertainty](https://arxiv.org/abs/2102.13640).  Moreover, their universality in $C(\mathbb{R}^d,\mathcal{G}_2)$ is known, and has been shown in [Corollary 4.7](https://arxiv.org/abs/2101.05390).

In [13]:
# %run Benchmarks_Model_Builder_Mean_Var.ipynb
exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1104s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished
100%|██████████| 100/100 [00:00<00:00, 1258.39it/s]

Infering Parameters for Deep Gaussian Network to train on!
Done Getting Parameters for Deep Gaussian Network!
Training Deep Gaussian Network!
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.1s finished


Epoch 1/5
13/13 [==============================] - 0s 2ms/step - loss: 0.5153 - mse: 0.5148 - mae: 0.5153 - mape: 7928170.0000
Epoch 2/5
13/13 [==============================] - 0s 1ms/step - loss: 0.5138 - mse: 0.5132 - mae: 0.5138 - mape: 7219673.0000
Epoch 3/5
13/13 [==============================] - 0s 1ms/step - loss: 0.5123 - mse: 0.5117 - mae: 0.5123 - mape: 6515195.0000
Epoch 4/5
13/13 [==============================] - 0s 1ms/step - loss: 0.5109 - mse: 0.5101 - mae: 0.5109 - mape: 5863837.0000
Epoch 5/5
3/3 [==============================] - 0s 910us/step


...

Training Deep Gaussian Network!: END
#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


  0%|          | 0/20 [00:00<?, ?it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------------------------#
 Get Testing Errors for: Gaussian Models
#--------------------------------------#


100%|██████████| 20/20 [00:00<00:00, 1891.46it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
Training Results to date:
                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000016   0.000000  1.602694e-08   
W1                                0.000016   0.000000  1.602694e-08   
W1-95R                            0.000016   0.000000  1.602694e-08   
M-95L                             0.000136   0.000136  1.265976e-04   
M                                 0.000136   0.000136  1.265976e-04   
M-95R                             0.000136   0.000136  1.265976e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                             

In [14]:
print("Prediction Quality (Updated): Test")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Prediction Quality (Updated): Test
                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000016   0.000000  1.602694e-08   
W1                                0.000016   0.000000  1.602694e-08   
W1-95R                            0.000016   0.000000  1.602694e-08   
M-95L                             0.000136   0.000136  1.265976e-04   
M                                 0.000136   0.000136  1.265976e-04   
M-95R                             0.000136   0.000136  1.265976e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                                     KRidge          GBRF         DNN  \
W1-95L                         1.580490e-08  1.093142e-08    0.002389   
W1                             1.580490e-08  1.149926e-07    0.002797   
W1-95R                         1.58

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN
W1-95L,0.000016,0.000000,1.602694e-08,1.580490e-08,1.093142e-08,0.002389,0.000009,0.904181
W1,0.000016,0.000000,1.602694e-08,1.580490e-08,1.149926e-07,0.002797,0.000010,0.957354
W1-95R,0.000016,0.000000,1.602694e-08,1.580490e-08,3.208878e-07,0.003277,0.000010,1.008979
M-95L,0.000136,0.000136,1.265976e-04,1.257175e-04,9.070589e-05,0.045869,0.000000,0.006284
M,0.000136,0.000136,1.265976e-04,1.257175e-04,1.967334e-04,0.051925,0.000000,0.009059
M-95R,0.000136,0.000136,1.265976e-04,1.257175e-04,3.444595e-04,0.056059,0.000000,0.011467
N_Par,1335.000000,0.000000,2.000000e+02,0.000000e+00,1.109000e+05,521.000000,0.000000,521.000000
Train_Time,5.530573,0.855534,1.619880e+09,5.926316e-01,8.619890e-01,5.066507,0.413286,1.994012
Test_Time/MC-Oracle_Test_Time,0.461736,1.000000,4.379825e-04,3.304335e-03,5.246057e-03,0.344872,0.003471,0.313234


In [15]:
print("Prediction Quality (Updated): Train")
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

Prediction Quality (Updated): Train
                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000020   0.000000  1.602694e-08   
W1                                0.000035   0.000000  1.496441e-05   
W1-95R                            0.000078   0.000000  4.486117e-05   
M-95L                             0.000170   0.000168  1.265976e-04   
M                                 0.000295   0.000295  2.506632e-04   
M-95R                             0.000543   0.000664  4.987944e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                                     KRidge          GBRF         DNN  \
W1-95L                         1.580490e-08  6.791680e-08    0.002347   
W1                             1.496441e-05  1.365428e-05    0.002520   
W1-95R                         4.4

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN
W1-95L,0.000020,0.000000,1.602694e-08,1.580490e-08,6.791680e-08,0.002347,0.000010,1.004355
W1,0.000035,0.000000,1.496441e-05,1.496441e-05,1.365428e-05,0.002520,0.000023,1.028649
W1-95R,0.000078,0.000000,4.486117e-05,4.486162e-05,4.078659e-05,0.002712,0.000050,1.057973
M-95L,0.000170,0.000168,1.265976e-04,1.257175e-04,1.616148e-04,0.046939,0.000000,0.008983
M,0.000295,0.000295,2.506632e-04,2.498008e-04,2.233496e-04,0.049023,0.000127,0.010087
M-95R,0.000543,0.000664,4.987944e-04,4.979672e-04,3.150803e-04,0.050634,0.000380,0.011377
N_Par,1335.000000,0.000000,2.000000e+02,0.000000e+00,1.109000e+05,521.000000,0.000000,521.000000
Train_Time,5.530573,0.855534,1.619880e+09,5.926316e-01,8.619890e-01,5.066507,0.413286,1.994012
Test_Time/MC-Oracle_Test_Time,0.461736,1.000000,4.379825e-04,3.304335e-03,5.246057e-03,0.344872,0.003471,0.313234


# 3) The natural Universal Benchmark: [Bishop's Mixture Density Network](https://publications.aston.ac.uk/id/eprint/373/1/NCRG_94_004.pdf)

This implementation is as follows:
- For every $x$ in the trainingdata-set we fit a GMM $\hat{\nu}_x$, using the [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), with the same number of centers as the deep neural model in $\mathcal{NN}_{1_{\mathbb{R}^d},\mathcal{D}}^{\sigma:\star}$ which we are evaluating.  
- A Mixture density network is then trained to predict the infered parameters; given any $x \in \mathbb{R}^d$.

In [16]:
if output_dim == 1:
    # %run Mixture_Density_Network.ipynb
    exec(open('Mixture_Density_Network.py').read())

  0%|          | 0/100 [00:00<?, ?it/s]

Preparing Training Outputs for MDNs using EM-Algorithm


100%|██████████| 100/100 [00:24<00:00,  4.01it/s]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Prepared Training Outputs for MDNs using EM-Algorithm!
Deep Feature Builder - Ready
(0)
Training Mixture Density Network (MDN): Means: Start!
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.3s finished


...

Epoch 1/5
13/13 [==============================] - 0s 1ms/step - loss: 4.2151 - accuracy: 0.0000e+00
Epoch 2/5
13/13 [==============================] - 0s 2ms/step - loss: 3.9320 - accuracy: 0.1900
Epoch 3/5
13/13 [==============================] - 0s 2ms/step - loss: 3.5633 - accuracy: 0.8500
Epoch 4/5
13/13 [==============================] - 0s 2ms/step - loss: 3.0666 - accuracy: 1.0000
Epoch 5/5
3/3 [==============================] - 0s 2ms/step


  0%|          | 0/100 [00:00<?, ?it/s]

Training Mixture Density Network (MDN): Mixture Coefficients: END!
#--------------------#
 Get Training Error(s)
#--------------------#


100%|██████████| 20/20 [00:00<00:00, 161.77it/s]


#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------#
 Get Test Error(s)
#--------------------#
#---------------------#
 Get Test Error(s): END
#---------------------#
#---------------------------#
 Get Training Error(s): Begin
#---------------------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------------#
 Get Testing Error(s): Begin
#--------------------------#
#------------------------#
 Get Testing Error(s): END
#------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
Updated DataFrame
-------------------------------------------------
Train
                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000020   0.000000  1.602694e-08   
W1                                0.000035   0.000000  1.496441e-05   


## Get Final Outputs
Now we piece together all the numerical experiments and report a nice summary.

# Result(s)

## Prediction Quality

#### Training

In [17]:
print("Final Training-Set Result(s)")
Summary_pred_Qual_models

Final Test-Set Result(s)


,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN,MDN
W1-95L,0.000020,0.000000,1.602694e-08,1.580490e-08,6.791680e-08,0.002347,0.000010,1.004355,0.986407
W1,0.000035,0.000000,1.496441e-05,1.496441e-05,1.365428e-05,0.002520,0.000023,1.028649,0.990409
W1-95R,0.000078,0.000000,4.486117e-05,4.486162e-05,4.078659e-05,0.002712,0.000050,1.057973,0.995872
M-95L,0.000170,0.000168,1.265976e-04,1.257175e-04,1.616148e-04,0.046939,0.000000,0.008983,0.012089
M,0.000295,0.000295,2.506632e-04,2.498008e-04,2.233496e-04,0.049023,0.000127,0.010087,0.013896
M-95R,0.000543,0.000664,4.987944e-04,4.979672e-04,3.150803e-04,0.050634,0.000380,0.011377,0.015976
N_Par,1335.000000,0.000000,2.000000e+02,0.000000e+00,1.109000e+05,521.000000,0.000000,521.000000,4005.000000
Train_Time,5.530573,0.855534,1.619880e+09,5.926316e-01,8.619890e-01,5.066507,0.413286,1.994012,0.166594
Test_Time/MC-Oracle_Test_Time,0.461736,1.000000,4.379825e-04,3.304335e-03,5.246057e-03,0.344872,0.003471,0.313234,238.679188


#### Test

In [18]:
print("Final Test-Set Result(s)")
Summary_pred_Qual_models_test

Final Training-Set Result(s)


,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN,MDN
W1-95L,0.000016,0.000000,1.602694e-08,1.580490e-08,1.093142e-08,0.002389,0.000009,0.904181,0.978482
W1,0.000016,0.000000,1.602694e-08,1.580490e-08,1.149926e-07,0.002797,0.000010,0.957354,0.990764
W1-95R,0.000016,0.000000,1.602694e-08,1.580490e-08,3.208878e-07,0.003277,0.000010,1.008979,1.002958
M-95L,0.000136,0.000136,1.265976e-04,1.257175e-04,9.070589e-05,0.045869,0.000000,0.006284,0.010325
M,0.000136,0.000136,1.265976e-04,1.257175e-04,1.967334e-04,0.051925,0.000000,0.009059,0.013564
M-95R,0.000136,0.000136,1.265976e-04,1.257175e-04,3.444595e-04,0.056059,0.000000,0.011467,0.018356
N_Par,1335.000000,0.000000,2.000000e+02,0.000000e+00,1.109000e+05,521.000000,0.000000,521.000000,4005.000000
Train_Time,5.530573,0.855534,1.619880e+09,5.926316e-01,8.619890e-01,5.066507,0.413286,1.994012,0.166594
Test_Time/MC-Oracle_Test_Time,0.461736,1.000000,4.379825e-04,3.304335e-03,5.246057e-03,0.344872,0.003471,0.313234,238.679188


# For Terminal Runner(s):

In [19]:
# For Terminal Running
print("============================")
print("Training Predictive Quality:")
print("============================")
print(Summary_pred_Qual_models)
print(" ")
print(" ")
print(" ")
print("===========================")
print("Testing Predictive Quality:")
print("===========================")
print(Summary_pred_Qual_models_test)
print("================================")
print(" ")
print(" ")
print(" ")
print("Kernel_Used_in_GPR: "+str(GPR_trash.kernel))
print("🙃🙃 Have a wonderful day! 🙃🙃")

Training Predictive Quality:
                                       DNM  MC-Oracle          ENET  \
W1-95L                            0.000020   0.000000  1.602694e-08   
W1                                0.000035   0.000000  1.496441e-05   
W1-95R                            0.000078   0.000000  4.486117e-05   
M-95L                             0.000170   0.000168  1.265976e-04   
M                                 0.000295   0.000295  2.506632e-04   
M-95R                             0.000543   0.000664  4.987944e-04   
N_Par                          1335.000000   0.000000  2.000000e+02   
Train_Time                        5.530573   0.855534  1.619880e+09   
Test_Time/MC-Oracle_Test_Time     0.461736   1.000000  4.379825e-04   

                                     KRidge          GBRF         DNN  \
W1-95L                         1.580490e-08  6.791680e-08    0.002347   
W1                             1.496441e-05  1.365428e-05    0.002520   
W1-95R                         4.486162e-

---
# Fin
---

---